### Read Thai City

In [ ]:
#select ชื่อเมืองในไทย
import json

with open('city.list.json', 'r') as f:
    d_test = json.load(f)

th_cities = [city for city in d_test if city['country'] == 'TH']
th_cities[:5]

#count = len(th_cities)
#count

[{'id': 1116760,
  'name': 'Don Sak',
  'state': '',
  'country': 'TH',
  'coord': {'lon': 99.691841, 'lat': 9.31676}}]

In [10]:
provinces = {
    city['name']: {
        'lat': city['coord']['lat'],
        'lon': city['coord']['lon']
    } for city in th_cities
}
# provinces
dict(list(provinces.items())[:5])

{'Don Sak': {'lat': 9.31676, 'lon': 99.691841},
 'Ban Thung That': {'lat': 8.26667, 'lon': 99.583328},
 'Thung Sai': {'lat': 16.31489, 'lon': 99.832672},
 'Tham Phannara': {'lat': 8.42044, 'lon': 99.395172},
 'Ban Talat Yai': {'lat': 7.88481, 'lon': 98.400078}}

### Fetch weather Data

In [ ]:
# Test Find Lat Lon
province = "Pathum Thani"
print(
    provinces[province]['lat'],
    provinces[province]['lon']
)

14.01346 100.530487


In [ ]:
#Endpoint API
WEATHER_ENDPOINT = "https://api.openweathermap.org/data/2.5/weather"
API_KEY = "68c16da5e675ad8635df84629765b118"

In [ ]:
import requests
import pandas as pd
from datetime import datetime
import time
import pytz

# # Endpoint API
# WEATHER_ENDPOINT = "https://api.openweathermap.org/data/2.5/weather"
# API_KEY = "68c16da5e675ad8635df84629765b118"


# Function to fetch and process weather data

def get_weather_data(province='Pathum Thani'):
    #Sleep
    # time.sleep(2)
    #ENDPOINT API Parameters
    WEATHER_ENDPOINT = "https://api.openweathermap.org/data/2.5/weather"
    API_KEY = "68c16da5e675ad8635df84629765b118"
    params = {
        "lat": provinces[province]['lat'],
        "lon": provinces[province]['lon'],
        "appid": API_KEY,
        "units": "metric"
    }

    try:
        # Make API request
        response = requests.get(WEATHER_ENDPOINT, params=params)
        response.raise_for_status()  # Raise an exception for bad status codes
        data = response.json()
        # return data # Fetch Weather Data
        
        # Convert timestamp to datetime
        dt = datetime.now()
        thai_tz = pytz.timezone('Asia/Bangkok')
        created_at = dt.replace(tzinfo=thai_tz)
        
        timestamp = datetime.now()
        
        # Create dictionary with required fields
        weather_dict = {
            'timestamp': timestamp,
            'year': timestamp.year,
            'month': timestamp.month,
            'day': timestamp.day,
            'hour': timestamp.hour,
            'minute': timestamp.minute,
            'created_at': created_at,
            'requested_province' : province,
            'location': data['name'],
            'weather_main': data['weather'][0]['main'],
            'weather_description': data['weather'][0]['description'],
            'main.temp': data['main']['temp']
        }
        
        # Create DataFrame
        # df = pd.DataFrame([weather_dict])
        
        # return df
        return weather_dict
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None
    except KeyError as e:
        print(f"Error processing data: Missing key {e}")
        return None
    

In [15]:
get_weather_data()

{'timestamp': datetime.datetime(2025, 4, 26, 2, 8, 43, 669855),
 'year': 2025,
 'month': 4,
 'day': 26,
 'hour': 2,
 'minute': 8,
 'created_at': datetime.datetime(2025, 4, 26, 2, 8, 43, 628633, tzinfo=<DstTzInfo 'Asia/Bangkok' LMT+6:42:00 STD>),
 'requested_province': 'Pathum Thani',
 'location': 'Pathum Thani',
 'weather_main': 'Clouds',
 'weather_description': 'few clouds',
 'main.temp': 29.99}

### FLOW

In [23]:
weatherdata_df = pd.DataFrame([get_weather_data(p) for p in list(provinces.keys())[:2]])
weatherdata_df

,timestamp,year,month,day,hour,minute,created_at,requested_province,location,weather_main,weather_description,main.temp
0,2025-04-26 02:17:25.524864,2025,4,26,2,17,2025-04-26 02:35:25.524837+07:00,Don Sak,Don Sak,Clouds,overcast clouds,27.17
1,2025-04-26 02:17:25.641887,2025,4,26,2,17,2025-04-26 02:35:25.641859+07:00,Ban Thung That,Ban Thung That,Clear,clear sky,24.42
